In [1]:
import pickle
import pandas as pd
import re

with open("paras.pkl", "rb") as pk:
    pkl = pickle.load(pk)

In [2]:
df = pd.DataFrame(list(pkl.items()), columns=['html_file', 'section'])

In [3]:
df['access_no'] = df['html_file'].replace(r"(.+_)(.+)(\.html)", r"\2", regex=True)
df['ticker'] = df['html_file'].replace(r"(.+)(_.+)", r"\1", regex=True)

In [4]:
df.head()

,html_file,section,access_no,ticker
0,MSM_0001003078-17-000307.html,"During fiscal 1999, the Board of Directors est...",0001003078-17-000307,MSM
1,RPAI_0001222840-18-000008.html,Table of Contents significant re-leasing activ...,0001222840-18-000008,RPAI
2,VSTO_0001616318-17-000102.html,Table of Contents The Credit Agreement and the...,0001616318-17-000102,VSTO
3,AWI_0001564590-18-003128.html,"On July 29, 2016, the Company announced that i...",0001564590-18-003128,AWI
4,BKD_0001332349-18-000033.html,"On November 1, 2016, the Company announced tha...",0001332349-18-000033,BKD


In [5]:
date = re.compile(r"(([A-Z][a-z]+ \d{1,2}, \d{4}))|" +
                   "((\d{1,2} [A-Z][a-z]+, \d{4}))|" +
                   "( (\d{4}))")

values = re.compile(r"( (\d[0-9.]+ [mb]illions?))|" +
                   "( (\d{1,3}(,\d{1,3})*) [A-Za-z]+s)|" +
                   "( (\$\d[0-9.]+ [mb]illions?))|" +
                   "( ($\d{1,3}(,\d{1,3})*) )")

context = r"([A-Za-z,]+)? ?([A-Za-z,]+)? ?([A-Za-z,]+)? ?{} ?([A-Za-z,]+)? ?([A-Za-z,]+)? ?([A-Za-z,]+)?"

In [6]:
cats = ["Share Repurchase Authorization Date",
"Share Repurchase Authorization",
"Share Repurchase Intention",
"Share Repurchase Count",
"Amount Spent on Share Repurchases",
"Share Repurchase Utilization",
"Unknown Share Repurchase Data"]

In [7]:
auth = ["auth", "repurchase "]
intent = ["plan", "intend", "repurchased"]
count = ["stock", "share"]
utili = ["utiliz", "allocat"]

In [8]:
cols = df.columns.tolist()
cols.extend(['year', 'quantity'])

final = pd.DataFrame([])

In [9]:
from copy import deepcopy

empty = []
for index, row in df.iterrows():
    para = row.section
    flag = 0
    sents = para.split(". ")
    for sent in sents:
        if values.search(sent) and "repurchas" in sent:
            
            flag = 1
            
            vs = values.findall(sent)
            ds = date.findall(sent)
            
            for g in vs:
                g = list(g)
                while '' in g:
                    g.remove('')
                v = g[1]
                
                line = deepcopy(list(row))
                line.append(v)
                line.append(sent)  
                
                if '$' in v:
                    c = context.format('\\' + v)
                else:
                    c = context.format(v)
                    
                    
                m = re.search(c, sent).group()
                
                flag2 = 0
                
                if "$" in v and "ed " in m:
                    final = final.append([line + [cats[4]]])

                for item in auth:
                    if item in m:
                        flag2 = 1
                        final = final.append([line + [cats[1]]])
                        
                for item in intent:
                    if item in m:
                        flag2 = 1
                        final = final.append([line + [cats[2]]])
                        
                for item in count:
                    if item in m:
                        flag2 = 1
                        final = final.append([line + [cats[3]]])
                
                for item in utili:
                    if item in m:
                        flag2 = 1
                        final = final.append([line + [cats[5]]])
                        
                if not flag2:
                    final = final.append([line + [cats[6]]])
                
                
            for j in ds:
                j = list(j)
                while '' in j:
                    j.remove('')
                d = j[1]
                
                line = deepcopy(list(row))
                line.append(d)
                line.append(sent)
                line.append(cats[0])
                final = final.append([line])
            
    if not flag:
        empty.append(row.html_file)

In [10]:
final.to_csv("xsub.csv")

In [11]:
final

,0,1,2,3,4,5,6
0,MSM_0001003078-17-000307.html,"During fiscal 1999, the Board of Directors est...",0001003078-17-000307,MSM,"5,000","In 2011, the Board of Directors reaffirmed and...",Share Repurchase Authorization
0,MSM_0001003078-17-000307.html,"During fiscal 1999, the Board of Directors est...",0001003078-17-000307,MSM,"5,000","In 2011, the Board of Directors reaffirmed and...",Share Repurchase Count
0,MSM_0001003078-17-000307.html,"During fiscal 1999, the Board of Directors est...",0001003078-17-000307,MSM,2011,"In 2011, the Board of Directors reaffirmed and...",Share Repurchase Authorization Date
0,MSM_0001003078-17-000307.html,"During fiscal 1999, the Board of Directors est...",0001003078-17-000307,MSM,802,"As of September 2, 2017, the maximum number of...",Share Repurchase Count
0,MSM_0001003078-17-000307.html,"During fiscal 1999, the Board of Directors est...",0001003078-17-000307,MSM,"September 2, 2017","As of September 2, 2017, the maximum number of...",Share Repurchase Authorization Date
0,MSM_0001003078-17-000307.html,"During fiscal 1999, the Board of Directors est...",0001003078-17-000307,MSM,685,"During fiscal 2017 and 2016, the Company repur...",Share Repurchase Intention
0,MSM_0001003078-17-000307.html,"During fiscal 1999, the Board of Directors est...",0001003078-17-000307,MSM,685,"During fiscal 2017 and 2016, the Company repur...",Share Repurchase Count
0,MSM_0001003078-17-000307.html,"During fiscal 1999, the Board of Directors est...",0001003078-17-000307,MSM,"5,344","During fiscal 2017 and 2016, the Company repur...",Share Repurchase Count
0,MSM_0001003078-17-000307.html,"During fiscal 1999, the Board of Directors est...",0001003078-17-000307,MSM,2017,"During fiscal 2017 and 2016, the Company repur...",Share Repurchase Authorization Date
0,MSM_0001003078-17-000307.html,"During fiscal 1999, the Board of Directors est...",0001003078-17-000307,MSM,2016,"During fiscal 2017 and 2016, the Company repur...",Share Repurchase Authorization Date
